A couple graphs with several key time series for jobs reports. Update whenver a new one releases.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils as ut

from plotly.subplots import make_subplots

In [2]:
from fredapi import Fred

API_KEY_PATH = "fred_api_key.txt" 
fred = Fred(api_key_file = API_KEY_PATH)

In [3]:
import graph_templates

pio.templates.default = 'fed_2025'

# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "jobs_report"

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = "../graph_output"


4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [4]:
# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "Nonfarm Payrolls": "PAYEMS",
    "U-6 Unemployment Rate": "U6RATE",
    "Unemployment Rate": "UNRATE",
    "Average Hourly Earnings": "CES0500000003",
}
today = dt.date.today()
all_start_date = dt.date(2022, 1, 1) # Start all on same date for subplot alignment

In [5]:
payrolls_srs = fred.get_series(
    series_id=series_codes["Nonfarm Payrolls"],
    observation_start=all_start_date,
    observation_end=today,
    frequency='m',
    units='chg' # change from previous observation
).rename("Change in Nonfarm Payrolls")

u3_srs = fred.get_series(
    series_id=series_codes["Unemployment Rate"],
    observation_start=all_start_date,
    observation_end=today,
).rename("Unemployment Rate")

u6_srs = fred.get_series(
    series_id=series_codes["U-6 Unemployment Rate"],
    observation_start=all_start_date,
    observation_end=today,
).rename("U6 Unemployment Rate")

earnings_srs = fred.get_series(
    series_id=series_codes["Average Hourly Earnings"],
    observation_start=all_start_date,
    observation_end=today,
    units='pc1' # percent change from previous observation
).rename("Percent Change in Average Hourly Earnings (YoY)")

joined_unemp_df = pd.concat(
    [u6_srs, u3_srs], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # include observations where some of the 3 are missing
)

# Naming the index makes it easier to title
joined_unemp_df.index.name = "Date"

joined_unemp_df.tail()

,U6 Unemployment Rate,Unemployment Rate
Date,,
2025-04-01,7.8,4.2
2025-05-01,7.8,4.2
2025-06-01,7.7,4.1
2025-07-01,7.9,4.2
2025-08-01,8.1,4.3


In [6]:
# Create a figure with 3 vertically stacked subplots (shared x-axis)

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    subplot_titles=(
        "Unemployment Rates",
        "Change in Nonfarm Payrolls",
        "Percent Change in Average Hourly Earnings (YoY)"
    )
)

# 1. Plot both unemployment rates in the first subplot
for col in joined_unemp_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_unemp_df.index,
            y=joined_unemp_df[col],
            mode='lines',
            name=col
        ),
        row=1, col=1
    )

# 2. Plot payrolls_srs in the second subplot
fig.add_trace(
    go.Scatter(
        x=payrolls_srs.index,
        y=payrolls_srs.values,
        mode='lines',
        name=payrolls_srs.name
    ),
    row=2, col=1
)

# 3. Plot earnings_srs in the third subplot
fig.add_trace(
    go.Scatter(
        x=earnings_srs.index,
        y=earnings_srs.values,
        mode='lines',
        name=earnings_srs.name
    ),
    row=3, col=1
)

# Update layout and axes
fig.update_layout(
    height=900,
    title=dict(
        text='Jobs Report<br><sup>Monthly, Seasonally Adjusted</sup>'
    ),
    showlegend=True,
    xaxis3_title="Date"
)

# Y-axis formatting for each subplot
fig.update_yaxes(
    title_text="Percent", tickformat=".0f", ticksuffix="%", row=1, col=1
)
fig.update_yaxes(
    title_text="Thousands", tickformat=",0", ticksuffix="k", row=2, col=1
)
fig.update_yaxes(
    title_text="Percent Change", tickformat=".1f", ticksuffix="%", row=3, col=1
)

# X-axis formatting (bottom subplot only)
fig.update_xaxes(
    type='date',
    tickformat='%b %Y',
    row=3, col=1
)


fig.update_layout(
    legend=dict(
        font_size=16,
        orientation='v'
    )
)

fig.show()

# Save the figure as HTML
fig.write_html(GRAPH_OUTPUT_PATH + f"/{GRAPH_NAME}.html")